In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [3]:
with open('/kaggle/input/arab-data/all_data.txt', 'r',encoding="utf-8") as f:
    text = f.read()

In [4]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [6]:
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[61 28 15  6 15 48 15 12 39  6]
 [25 45  6  2 12  6 54 12 39 15]
 [32 45 54 45 68 25 12  6 35 45]
 [13 62 78  6 12 39  0 78 50 68]
 [35 45 80 45 25 21 45 80 68 12]
 [80 13 60 19  6 74 45 25 49 12]
 [43 12 25 78 43 12  1  1 54 15]
 [68 43 16 13 15 43  6 39 12  6]]

y
 [[28 15  6 15 48 15 12 39  6 35]
 [45  6  2 12  6 54 12 39 15 19]
 [45 54 45 68 25 12  6 35 45 80]
 [62 78  6 12 39  0 78 50 68 25]
 [45 80 45 25 21 45 80 68 12  6]
 [13 60 19  6 74 45 25 49 12 59]
 [12 25 78 43 12  1  1 54 15 70]
 [43 16 13 15 43  6 39 12  6 15]]


In [8]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [9]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [10]:
def train(net, data, epochs=5, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    valid_loss_min = np.Inf # track change in validation loss


    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = train_loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = valid_loss =  criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        model_name = 'best_loss_so_far.net'

        checkpoint = {'n_hidden': net.n_hidden,
                       'n_layers': net.n_layers,
                       'state_dict': net.state_dict(),
                       'tokens': net.chars}

        with open(model_name, 'wb') as f:
            torch.save(model_name, f)

In [11]:
n_hidden=512
n_layers=5

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(82, 512, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=82, bias=True)
)


In [12]:
batch_size = 128
seq_length = 100
n_epochs = 10 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/10... Step: 10... Loss: 3.3892... Val Loss: 3.3127
Epoch: 1/10... Step: 20... Loss: 3.3567... Val Loss: 3.2767
Epoch: 1/10... Step: 30... Loss: 3.3429... Val Loss: 3.2799
Epoch: 1/10... Step: 40... Loss: 3.3365... Val Loss: 3.2785
Epoch: 1/10... Step: 50... Loss: 3.3218... Val Loss: 3.2701
Epoch: 1/10... Step: 60... Loss: 3.3108... Val Loss: 3.2726
Epoch: 1/10... Step: 70... Loss: 3.3240... Val Loss: 3.2727
Epoch: 1/10... Step: 80... Loss: 3.3202... Val Loss: 3.2741
Epoch: 1/10... Step: 90... Loss: 3.3168... Val Loss: 3.2797
Epoch: 1/10... Step: 100... Loss: 3.3222... Val Loss: 3.2733
Epoch: 1/10... Step: 110... Loss: 3.3245... Val Loss: 3.2799
Epoch: 1/10... Step: 120... Loss: 3.3083... Val Loss: 3.2724
Epoch: 1/10... Step: 130... Loss: 3.3037... Val Loss: 3.2783
Epoch: 1/10... Step: 140... Loss: 3.3027... Val Loss: 3.2739
Epoch: 1/10... Step: 150... Loss: 3.2983... Val Loss: 3.2748
Epoch: 1/10... Step: 160... Loss: 3.3006... Val Loss: 3.2779
Epoch: 1/10... Step: 170... Loss:

In [13]:
with open('/kaggle/input/temprnn/Temp.net', 'rb') as f:
    if train_on_gpu:
            checkpoint = torch.load(f)
    else:        
        checkpoint = torch.load(f,map_location=torch.device('cpu'))

    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [14]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [15]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [16]:
print(sample(loaded, 300, prime='انا', top_k=5))


انا

في الحب من ماء ابن حيّ أعين

وأعلى بعد العما ما تعدلا

فتن الموت والنافسُ أن يرى

لله من أوردت له ما انفضا

فاترك الأمر من المرء معا

لا تحيا بعينه المستطارا

في كل قلب للمنايا النابي

فالمرء من من لم تزد بالحقاما

والله ما أنف المرام الأنام

فما لها من كل أرض أبي

وأحب من أمر الحياة والراح

فلين م
